In [ ]:
# script to replace the base mechanism with new reactions
# Includes every reaction and species calculated above the uncertainty threshold

In [1]:
import os
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model
basedir = '../models/base_rmg'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')

# load the uncorrelated uncertainty
kinetic_uncertainty = np.load(os.path.join(basedir, 'kinetic_uncertainty.npy'))
thermo_uncertainty = np.load(os.path.join(basedir, 'thermo_uncertainty.npy'))

110 species, 1822 reactions


In [3]:
# Load the new kinetics library
kinetics_lib = '../kinetics'
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
print(f'{len(ark_kinetics_database.libraries[""].entries)} kinetic entries loaded')


# Load the new thermo library
thermo_lib = '../thermo'
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} thermo entries loaded')

105 kinetic entries loaded
110 thermo entries loaded


In [4]:
# Figure out which reactions to change
uncertainty_cutoff = 3.0
change_list = []
assert len(kinetic_uncertainty) == len(reaction_list)
for i in range(0, len(reaction_list)):
    if kinetic_uncertainty[i] > uncertainty_cutoff:
        change_list.append(i)

In [6]:
# Change the kinetics for the specified reactions
total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
    
    if rmg_rxn_index in change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

print(total_changed, 'changed')

changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 418 HO2(16) + CH3CO(20) <=> H2O2(17) + CH2CO(24)
changing reaction 419 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24)
changing reaction 420 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8-2(189)
changing reaction 427 CH2CHO(21) + CH2CHO(21) <=> CH2CO(24) + CH3CHO(35)
changing reaction 574 CH3O(31) + C3H5-A(94) <=> CH2O(9) + C3H6(12)
changing reaction 581 CH2CHO(21) + C3H5-A(94) <=> CH2CO(24) + C3H6(12)
changing reaction 720 CH2CHO(21) + C4H71-3(190) <=> CH2CO(24) + C4H8-2(189)
changing reaction 722 CH2CHO(21) + C4H71-3(190) <=> CH2CO(24) + C4H8-1(188)
changing reaction 725 CH3O(31) + C4H71-3(190) <=> CH2O(9) + C4H8-2(189)
changing reaction 739 CH2CHO(21) + C4H8-1(188) <=> CH3CHO(35) + C4H71-3(190)
changing r

In [8]:
# Figure out which species to change
uncertainty_cutoff = 3.0
thermo_change_list = []
assert len(thermo_uncertainty) == len(species_list)
for i in range(0, len(species_list)):
    if thermo_uncertainty[i] > uncertainty_cutoff:
        thermo_change_list.append(i)

In [9]:
# stitch each of the library thermo params into the mechanism

sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
    for i, species in enumerate(species_list):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
            continue
            raise ValueError (f'could not match species index {entry}')
        
        
    if rmg_species_index in thermo_change_list:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')   
    

Changing estimated thermo 80: [CH2]C[O](703)
Changing estimated thermo 81: [CH2]C[CH]C(748)
Changing estimated thermo 82: [CH2]CC(C)[O](766)
Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 91: [O]C[O](988)
Changing estimated thermo 93: [CH2]C=C[CH2](2534)
Changing estimated thermo 97: C=CC(C)O[O](4243)
Changing estimated thermo 98: CC=CCO[O](4244)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 100: C=CCCO[O](4407)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
12 changed


In [11]:
# Save the new mechanism
chemkin_file = os.path.join(f'cutoff3_20221205.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])


Wrote CTI mechanism file to 'cutoff3_20221205.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=cutoff3_20221205.inp', '--transport=../models/base_rmg/tran.dat', '--output=cutoff3_20221205.cti'], returncode=0)